In [ ]:
# ! python -m spacy download ru_core_news_lg
# ! pip install langdetect 
# ! pip install -U sentence-transformers

In [ ]:
import os
import calendar

import pandas as pd
import numpy as np 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import spacy
import torch
import torch.nn as nn
import re
import string
from langdetect import detect as detect_lang

from matplotlib import pyplot as plt
from nltk.corpus import stopwords
from nltk import tokenize
from wordcloud import WordCloud, STOPWORDS

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

import xgboost as xgb
from xgboost import XGBClassifier

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import seaborn as sns
from tqdm import tqdm

from copy import deepcopy
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay

import ru_core_news_lg
nlp = ru_core_news_lg.load()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Small dataset preprocessing

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Fake_news_detection/fake_detection_df_train.csv')
df_val = pd.read_csv('/content/drive/MyDrive/Fake_news_detection/fake_detection_df_val.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Fake_news_detection/fake_detection_df_test.csv')

In [ ]:
combined = [df_train, df_val, df_test]
pd.set_option('max_colwidth', 150)

df_train[['text', 'label']].loc[df_train['label'] == False]

,text,label
5,⚠️ ? Зафиксирован обстрел со стороны ВФУ по направлению: ▶️ 01:15 - н.п. Невельское - н.п. Лозовое: выпущено 3 мины калибром 120 мм;,False
7,Ми-35 ВКС России прикрывает колонну,False
8,? Ещё видео со зданием СБУ в Чернигове,False
9,"Блинкен в разговоре с Кулебой заявил, что США продолжат предоставлять Украине помощь для защиты от России, заявили в Госдепе.",False
10,⚡️ Эдуарб Басурин: Подразделения ДНР взяли под свой контроль населённые пункты Павлополь и Пищевик на Мариупольском направлении,False
...,...,...
863,"Глава МИД Германии заявила, что целью санкций ЕС является «международная и политическая изоляция российского руководства». @rt_russian",False
868,❗️ ?? Ещё российские войска и техника в Харькове.,False
869,"❗️ Зеленский заявил, что президенты Турции и Азербайджана предложили организовать переговоры с Россией. Зеленский заявил, что получил заверения в ...",False
871,"Спасибо, что смотрите и доверяете.",False


In [ ]:
def detect_lang_with_excep(input):
    try:
        return detect_lang(input)
    except:
        return "none"
        
for d in combined:
  d["lang"] = d["text"].apply(detect_lang_with_excep)
  print(d["lang"].value_counts())

ru    868
bg      7
uk      2
da      1
so      1
mk      1
Name: lang, dtype: int64
ru    123
ro      1
en      1
bg      1
et      1
Name: lang, dtype: int64
ru      242
uk        4
none      1
bg        1
Name: lang, dtype: int64


In [ ]:
#Here we create a function for text preprocessing.

nlp = spacy.load('ru_core_news_lg')

def remove_emojis(data):
    emoj = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def text_preprocessing(text):
    text = remove_emojis(text)
    russian_stop_words = stopwords.words('russian')
    lower_text = [i.lower() for i in text]
    custom_punct = string.punctuation +"«" + "»‎"
    no_latin = [i for i in lower_text if i not in list(string.ascii_letters)]
    no_punct = [i for i in no_latin if i not in list(custom_punct)]
    no_punct = ''.join(no_punct)
    no_sw = [i for i in no_punct.split() if i not in russian_stop_words]
    no_sw = ' '.join(no_sw)
    document = nlp(no_sw)
    result = " ".join([token.lemma_ for token in document])
    return result

In [ ]:
for d in combined:
  d['preprocessed_text'] = d['text'].apply(lambda x: text_preprocessing(x))

df_train

,id,text,label,lang,preprocessed_text
0,00000_03514,"Чернигов прилет во многоэтажку. Говорят русская ракета, но сейчас хз, город ведь под контролем РФ. Люди прячьтесь в подвалы, таких случаев может б...",True,ru,чернигов прилёт многоэтажка говорить русский ракета хз город контроль рф человек прятаться подвал такой случай
1,00002_06059,Председатель Следственного комитета РФ Александр Бастрыкин поручил проверить информацию об обстрелах украинскими силовиками станицы в Краснодарско...,True,ru,председатель следственный комитет рф александр бастрыкин поручить проверить информация обстрел украинскими силовик станица краснодарский край данн...
2,00003_08645,Все сейчас массово хотят уехать со Львова.,True,ru,массово хотеть уехать львов
3,00004_00901,"«К военным подошли бабушки и попросили убрать «Аллею славы героев Украины», сказали: «Мы ждали 8 лет вас, и дождались наконец». Ничто человеческое...",True,ru,военный подойти бабушка попросить убрать аллея слава герой украина сказать ждать 8 год дождаться ничто человеческий чуждо военный сразу сравнить ц...
4,00006_06251,"С уважение отношусь к Лобаеву, но Владислав, если у вас есть вопросы к «прославленным» корреспондентам ВГТРК, обращайтесь. Тем более, уверен, знае...",True,ru,уважение относиться лобаеву владислав вопрос прославленным корреспондент вгтрк обращаться уверенный знать выйти связь растолкую публичный истерика...
...,...,...,...,...,...
875,08891_03363,"?? ?? ? Появились цели захватить Москву, Чечню и Крым?. Обращение Адама Осмаева, командира чеченского батальона им Джохара Дудаева касательно бата...",True,ru,появиться цель захватить москва чечня крым обращение адам осмаев командир чеченский батальон джохар дудаев касательно батальон чеченский военнослу...
876,08892_01638,"В украинских пабликах пишут, что российский спецназ занял киевское метро. Подтвердить, так же как опровергнуть, пока сложно. Даже логически.",True,ru,украинских пабликах писать российский спецназ занять киевский метро подтвердить опровергнуть пока сложный логически
877,08893_03217,"Сообщается, что в дом попала ракета ПВО ВС Украины.",True,ru,сообщаться дом попасть ракета пво вс украина
878,08896_08921,"В Росавиации сообщили , что с сегодняшнего дня Россия ограничивает использование своего неба для британских самолетов, включая транзит, в ответ на...",True,ru,росавиации сообщить сегодняшний день россия ограничивать использование свой небо британский самолёт включая транзит ответ санкция лондон против аэ...


In [ ]:
for d in combined:
  print(d.shape)
  d.drop_duplicates(inplace=True)
  d.drop(d[d.lang != 'ru'].index, inplace=True)
  d.drop(columns = ['id', 'text', 'lang'], inplace = True)
  d['label'] = d['label'] * 1
  d.reset_index(drop = True, inplace = True)
  print(d.shape)

(880, 5)
(868, 2)
(127, 5)
(123, 2)
(248, 5)
(242, 2)


#Params

In [ ]:
param_count = {
        'vectorizer__max_df': [0.5, 0.75, 1.0],
        'vectorizer__max_features': [None, 5000, 10000, 50000],
        'vectorizer__ngram_range': [(1, 1), (1, 2), (1,3)]}

In [ ]:
param_tfidf = {'vectorizer__max_df': [0.5, 0.75, 1.0],
            'vectorizer__max_features': [None, 5000, 10000, 50000],
            'vectorizer__ngram_range': [(1, 1), (1, 2), (1,3)],
            'vectorizer__norm': ['l1', 'l2', None]}

In [ ]:
param_logreg = {'model__solver' : ['newton-cg', 'lbfgs', 'liblinear'],
        'model__C' : [100, 10, 1.0, 0.1, 0.01, 0.001],
        'model__max_iter': [10000]}

In [ ]:
param_nb = {'model__alpha': [0.0001, 0.001, 0.01, 1.0, 10, 100]}

In [ ]:
param_rf = {'model__max_features' : ['auto', 'sqrt'], 
            'model__max_depth' : [5,10,15],
            'model__bootstrap' : [True, False]}

In [ ]:
param_xgb = {'model__learning_rate': [0.1, 0.01],
              'model__n_estimators': [50]}

#Linear Models and Vectorizers

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

Xtrain = df_train['preprocessed_text']
ytrain = df_train['label']
Xtest = df_test['preprocessed_text']
ytest = df_test['label']


list_vect = [(CountVectorizer(), param_count), (TfidfVectorizer(), param_tfidf)]
list_models = [(LogisticRegression(), param_logreg), (MultinomialNB(), param_nb), (RandomForestClassifier(), param_rf), (xgb.XGBClassifier(), param_xgb)]

for v in list_vect:
    for m in list_models:
        pipeline = Pipeline([("vectorizer", v[0]), ("model", m[0])])
        dict_params = {}

        dict_params.update(v[1])
        dict_params.update(m[1])

        print('\nPipeline:', pipeline)
        print('Param Grid:', dict_params)

        GridSearch = GridSearchCV(pipeline, dict_params, scoring='roc_auc', n_jobs = -1, cv = 3)

        GridSearch.fit(Xtrain, ytrain)
        print(GridSearch.best_params_)

        train_preds = GridSearch.predict_proba(Xtrain)[:,1]
        test_preds = GridSearch.predict_proba(Xtest)[:,1]

        train_targets = ytrain.values
        test_targets = ytest.values

        print('Best Params:', GridSearch.best_params_)
        print(f'\n Combination of {v[0]} + {m[0]} gives:')
        print('\nTrain Roc Auc Score:', roc_auc_score(train_targets, train_preds))
        print('Test Roc Auc Score:', roc_auc_score(test_targets, test_preds))



Pipeline: Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('model', LogisticRegression())])
Param Grid: {'vectorizer__max_df': [0.5, 0.75, 1.0], 'vectorizer__max_features': [None, 5000, 10000, 50000], 'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)], 'model__solver': ['newton-cg', 'lbfgs', 'liblinear'], 'model__C': [100, 10, 1.0, 0.1, 0.01, 0.001], 'model__max_iter': [10000]}
{'model__C': 0.01, 'model__max_iter': 10000, 'model__solver': 'newton-cg', 'vectorizer__max_df': 0.5, 'vectorizer__max_features': None, 'vectorizer__ngram_range': (1, 3)}
Best Params: {'model__C': 0.01, 'model__max_iter': 10000, 'model__solver': 'newton-cg', 'vectorizer__max_df': 0.5, 'vectorizer__max_features': None, 'vectorizer__ngram_range': (1, 3)}

 Combination of CountVectorizer() + LogisticRegression() gives:

Train Roc Auc Score: 0.9838163554862159
Test Roc Auc Score: 0.7790406673618352

Pipeline: Pipeline(steps=[('vectorizer', CountVectorizer()), ('model', MultinomialNB())])
Param G

#Linear Models and Sentence Embeddings

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Fake_news_detection/fake_detection_df_train.csv')
df_val = pd.read_csv('/content/drive/MyDrive/Fake_news_detection/fake_detection_df_val.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Fake_news_detection/fake_detection_df_test.csv')

df_train = pd.concat([df_val, df_train], axis = 0)

combined = [df_train, df_test]

In [ ]:
# Here we create a function for light text preprocessing so that transformer can benefit from more text information.

def remove_emojis(data):
    emoj = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)


def light_preprocessing(text):
    text = remove_emojis(text)
    lower_text = [i.lower() for i in text]
    custom_punct = string.punctuation +"«" + "»‎"
    no_latin = [i for i in lower_text if i not in list(string.ascii_letters)]
    no_punct = [i for i in no_latin if i not in list(custom_punct)]
    no_punct = ''.join(no_punct)
    return no_punct

for d in combined:
  d['preprocessed_light'] = d['text'].apply(lambda x: light_preprocessing(x))

In [ ]:
for d in combined:
  print(d.shape)
  d.drop_duplicates(inplace=True)
  d.drop(columns = ['id', 'text'], inplace = True)
  d['label'] = d['label'] * 1
  d.reset_index(drop = True, inplace = True)
  print(d.shape)

(1007, 4)
(1007, 2)
(248, 4)
(248, 2)


In [ ]:
sent_tr = SentenceTransformer("DeepPavlov/rubert-base-cased", device="cpu")

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/DeepPavlov_rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_embs = sent_tr.encode(df_train["preprocessed_light"].to_list())
test_embs = sent_tr.encode(df_test["preprocessed_light"].to_list())

In [ ]:
param_logreg = {'model__solver' : ['newton-cg', 'lbfgs', 'liblinear'],
        'model__C' : [100, 10, 1.0, 0.1, 0.01, 0.001],
        'model__max_iter': [10000]}

In [ ]:
param_rf = {'model__max_features' : ['auto', 'sqrt'], 
            'model__max_depth' : [5,10,15],
            'model__bootstrap' : [True, False]}

In [ ]:
param_xgb = {'model__learning_rate': [0.1, 0.01],
              'model__n_estimators': [50]}

In [ ]:
Xtrain = train_embs
ytrain = df_train['label']
Xtest = test_embs
ytest = df_test['label']

list_models = [(LogisticRegression(), param_logreg), (RandomForestClassifier(), param_rf), (xgb.XGBClassifier(), param_xgb)]

for v in list_models:
        pipeline = Pipeline([("scaler", StandardScaler()),
            ("model", v[0])])

        print('\nPipeline:', pipeline)

        GridSearch = GridSearchCV(pipeline, v[1], scoring='roc_auc', n_jobs = -1, cv = 3)

        GridSearch.fit(Xtrain, ytrain)
        print(GridSearch.best_params_)

        train_preds = GridSearch.predict_proba(Xtrain)[:,1]
        test_preds = GridSearch.predict_proba(Xtest)[:,1]

        train_targets = ytrain.values
        test_targets = ytest.values

        print('Best Params:', GridSearch.best_params_)
        print(f'\n Model {v[0]} gives:')
        print('\nTrain Roc Auc Score:', roc_auc_score(train_targets, train_preds))
        print('Test Roc Auc Score:', roc_auc_score(test_targets, test_preds))


Pipeline: Pipeline(steps=[('scaler', StandardScaler()), ('model', LogisticRegression())])


/usr/local/lib/python3.8/dist-packages/scipy/optimize/_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.8/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


{'model__C': 0.001, 'model__max_iter': 10000, 'model__solver': 'newton-cg'}
Best Params: {'model__C': 0.001, 'model__max_iter': 10000, 'model__solver': 'newton-cg'}

 Model LogisticRegression() gives:

Train Roc Auc Score: 0.8500681346976993
Test Roc Auc Score: 0.801300244208303

Pipeline: Pipeline(steps=[('scaler', StandardScaler()),
                ('model', RandomForestClassifier())])
{'model__bootstrap': False, 'model__max_depth': 10, 'model__max_features': 'sqrt'}
Best Params: {'model__bootstrap': False, 'model__max_depth': 10, 'model__max_features': 'sqrt'}

 Model RandomForestClassifier() gives:

Train Roc Auc Score: 0.9999979099785982
Test Roc Auc Score: 0.8073724506633225

Pipeline: Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                 